# Azure ML Online Endpoint – Deploy/Update notebook (single-endpoint, single-deployment)

Cel: **podmiana deploymentu** na istniejącym Azure ML Online Endpoint tak, aby aplikacja trafiała w właściwy deployment (nagłówek `azureml-model-deployment`).

Notebook robi:
1. Konfiguracja i logowanie do workspace
2. Weryfikacja endpointu i istniejących deploymentów
3. **Hard reset**: usuń deployment (jeśli istnieje) – usuwa konflikty `StartUpdateDeploymentAsync`
4. Utworzenie środowiska (environment) i deploymentu
5. Ustawienie traffic allocation
6. Test `invoke` + podgląd logów

> Zastąp wartości w sekcji **CONFIG**. Nie zmieniaj nazw pól w payload – są dostosowane do Twojej aplikacji.


In [ ]:
# 0) (opcjonalnie) Zainstaluj zależności – uruchom raz w nowym środowisku
# !pip -q install "azure-ai-ml>=1.16.0" "azure-identity>=1.16.0" "azure-core>=1.30.0"
# !pip -q install "requests>=2.31.0"


## 1) CONFIG – UZUPEŁNIJ

In [ ]:
# === AML WORKSPACE ===
SUBSCRIPTION_ID = "<SUBSCRIPTION_ID>"
RESOURCE_GROUP  = "<RESOURCE_GROUP>"
WORKSPACE_NAME  = "<WORKSPACE_NAME>"

# === AML ONLINE ENDPOINT ===
ENDPOINT_NAME   = "<EXISTING_ONLINE_ENDPOINT_NAME>"

# === DEPLOYMENT NAME (MUSI PASOWAĆ DO NAGŁÓWKA Z APKI) ===
# Apka wysyła: azureml-model-deployment: albot-{modelLower}-model
DEPLOYMENT_NAME = "<albot-xxx-model>"

# === MODELNAME wysyłany przez apkę (do testów invoke) ===
APP_MODEL_NAME  = "<WheShe_or_other>"

# === DOCKER / ENV ===
IMAGE_URI       = "<YOUR_IMAGE_URI>"
INFERENCE_PORT  = 5001

# === Azure OpenAI ENV VARS (w kontenerze) ===
AZURE_OPENAI_ENDPOINT = "https://<your-aoai-resource>.openai.azure.com/"
AZURE_OPENAI_API_KEY  = "<your-aoai-key>"
AZURE_OPENAI_API_VERSION = "2024-12-01-preview"

# CHAT_DEPLOYMENT: NAZWA deploymentu w Azure OpenAI (Portal -> Azure OpenAI -> Deployments -> 'Deployment name')
CHAT_DEPLOYMENT = "<gpt-5-mini>"

# (opcjonalnie) embeddings, jeśli Twój score.py tego wymaga
AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT = "<text-embedding-3-small-or-other>"  # lub ""

# === Compute/instance ===
INSTANCE_TYPE = "Standard_DS3_v2"
INSTANCE_COUNT = 1


## 2) Połączenie z AML + sanity check endpointu

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.core.exceptions import ResourceNotFoundError
import json

credential = DefaultAzureCredential(exclude_interactive_browser_credential=False)
ml_client = MLClient(credential, SUBSCRIPTION_ID, RESOURCE_GROUP, WORKSPACE_NAME)

endpoint = ml_client.online_endpoints.get(ENDPOINT_NAME)
print("Endpoint:", endpoint.name)
print("State:", endpoint.provisioning_state, "| Auth:", endpoint.auth_mode)

print("\nDeployments on endpoint:")
for d in ml_client.online_deployments.list(endpoint_name=ENDPOINT_NAME):
    print("-", d.name, "| state:", d.provisioning_state)


## 3) Hard reset deploymentu (usuń jeśli istnieje) – usuwa Conflict/locki

In [ ]:
from azure.core.exceptions import HttpResponseError

def delete_deployment_if_exists():
    try:
        ml_client.online_deployments.get(name=DEPLOYMENT_NAME, endpoint_name=ENDPOINT_NAME)
    except ResourceNotFoundError:
        print("Deployment does not exist:", DEPLOYMENT_NAME)
        return

    print("Deleting deployment:", DEPLOYMENT_NAME)
    ml_client.online_deployments.begin_delete(
        name=DEPLOYMENT_NAME,
        endpoint_name=ENDPOINT_NAME
    ).result()
    print("Deleted.")

delete_deployment_if_exists()


## 4) Utwórz deployment (Managed Online Deployment)

In [ ]:
from azure.ai.ml.entities import ManagedOnlineDeployment, Environment

env = Environment(
    name=f"{DEPLOYMENT_NAME}-env",
    image=IMAGE_URI
)

env_vars = {
    "AZURE_OPENAI_ENDPOINT": AZURE_OPENAI_ENDPOINT,
    "AZURE_OPENAI_API_KEY": AZURE_OPENAI_API_KEY,
    "AZURE_OPENAI_API_VERSION": AZURE_OPENAI_API_VERSION,
    "CHAT_DEPLOYMENT": CHAT_DEPLOYMENT,
}

if AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT and AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT.strip():
    env_vars["AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT"] = AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT

deployment = ManagedOnlineDeployment(
    name=DEPLOYMENT_NAME,
    endpoint_name=ENDPOINT_NAME,
    environment=env,
    instance_type=INSTANCE_TYPE,
    instance_count=INSTANCE_COUNT,
    environment_variables=env_vars,
)

print("Creating/Updating deployment:", DEPLOYMENT_NAME)
poller = ml_client.online_deployments.begin_create_or_update(deployment)
dep = poller.result()
print("Provisioning state:", dep.provisioning_state)


## 5) Ustaw traffic allocation (100% do nowego deploymentu)

In [ ]:
endpoint = ml_client.online_endpoints.get(ENDPOINT_NAME)
endpoint.traffic = {DEPLOYMENT_NAME: 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()
print("Traffic:", ml_client.online_endpoints.get(ENDPOINT_NAME).traffic)


## 6) Debug: status + logi kontenera

In [ ]:
from azure.core.exceptions import HttpResponseError

def show_deployment_status_and_logs(lines=200):
    d = ml_client.online_deployments.get(name=DEPLOYMENT_NAME, endpoint_name=ENDPOINT_NAME)
    print("Deployment:", d.name)
    print("Provisioning:", d.provisioning_state)
    print("Instance:", d.instance_type, "x", d.instance_count)

    try:
        logs = ml_client.online_deployments.get_logs(
            name=DEPLOYMENT_NAME,
            endpoint_name=ENDPOINT_NAME,
            lines=lines
        )
        print("\n--- LOGS (tail) ---\n")
        print(logs)
    except HttpResponseError as e:
        print("Could not fetch logs:", e)

show_deployment_status_and_logs(lines=400)


## 7) Test invoke (payload jak w aplikacji)

In [ ]:
test_payload = {
    "document": {
        "contentDomain": {"byId": {}},
        "contextId": "test",
    },
    "model": APP_MODEL_NAME,
    "num_preds": 3
}

try:
    resp = ml_client.online_endpoints.invoke(
        endpoint_name=ENDPOINT_NAME,
        deployment_name=DEPLOYMENT_NAME,
        request_file=None,
        request_json=test_payload
    )
    print("Response:")
    print(resp)
except Exception as e:
    print("Invoke failed:", repr(e))
    show_deployment_status_and_logs(lines=600)


## 8) Walidacja formatu odpowiedzi (czy apka zobaczy `documentDemandPredictions`)

In [ ]:
def validate_response_schema(resp_text: str):
    try:
        obj = json.loads(resp_text)
    except Exception:
        print("Response is not JSON.")
        return False

    arr = None
    if isinstance(obj, dict):
        if "predictions" in obj and isinstance(obj["predictions"], dict):
            arr = obj["predictions"].get("documentDemandPredictions")
        if arr is None:
            arr = obj.get("documentDemandPredictions")
    ok = isinstance(arr, list)
    print("Has documentDemandPredictions array:", ok)
    if ok and arr:
        print("First item keys:", list(arr[0].keys()))
    return ok

# Użycie:
# validate_response_schema(resp)
